In [1]:
# Import Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Get Dataset from Kaggle
# Requires the kaggle package to download dataset. Uncomment the line below to install
# !pip3 install kaggle
import kaggle
# Requires kaggle.json (Kaggle API Key) placed in ~/.kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files("drgilermo/nba-players-stats", path="./data/", unzip=True)

In [3]:
stats = pd.read_csv("./data/Seasons_Stats.csv")
stats = stats.rename(columns={"Unnamed: 0" : "Index"})
stats.drop(['blanl', 'blank2'], axis=1, inplace=True)
for index, row in stats.iterrows():
    if not pd.isna(row["Pos"]):
        positions = row["Pos"].split("-")
        # print(positions)
        stats.at[index, "Pos"] = positions[0]
        for i in range(1, len(positions)):
            position = positions[i]
            new_row = stats.iloc[index].copy()
            new_row["Pos"] = position
            stats.loc[len(stats)] = new_row
stats.dropna(inplace=True)

In [4]:
def height_to_inches(height):
    feet, inches = map(int, height.split("-"))
    return feet * 12 + inches

player_data = pd.read_csv("./data/player_data.csv")
player_data.dropna(inplace=True)

# Apply the function to the DataFrame
player_data["height_inches"] = player_data["height"].apply(height_to_inches)

# Add height and weight to players
stats['height_inches'] = np.nan
stats['weight'] = np.nan
for index, row in player_data.iterrows():
    stats.loc[stats['Player'] == row['name'], 'height_inches'] = row['height_inches']
    stats.loc[stats['Player'] == row['name'], 'weight'] = row['weight']

# Drop records with missing data
stats.dropna(inplace=True)

# Also dropping the name of the player...hopefully players aren't assigned positions based on their names
# Also dropping the year since we are not predicting the position of a player in a specific year
stats.drop(['Year', 'Player'], axis=1, inplace=True)
stats.head(5)

stats.to_csv("season_stats_preprocessed.csv")